In [1]:
import os

In [2]:
os.environ["OPENBLAS_NUM_THREADS"] = "1"  # For implicit ALS

In [3]:
!pip install rectools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.0/89.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.0/89.0 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 40.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 25.9 MB/s eta 0:00:00
  Created wheel for implicit: filename=implicit-0.4.4-cp39-cp39-linux_x86_64.whl size=5054334 sha256=5295fdf4e405999f2445d448f98ba38ef24ec3a2b0f844923aeef7c86be82d78
  Stored in directory: /root/.cache/pip/wheels/05/d3/5d/ab371b92ddb01183c3997751ad3bed6e6a

In [4]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install hyperopt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!python3 --version

Python 3.9.16


In [8]:
from random import randint

import pandas as pd
import numpy as np

from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from implicit.als import AlternatingLeastSquares

from lightfm import LightFM

In [9]:
!wget https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip

--2023-04-24 18:54:44--  https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78795385 (75M) [application/zip]
Saving to: ‘kion_train.zip’

kion_train.zip      100%[===================>]  75.14M  10.6MB/s    in 8.9s    

2023-04-24 18:54:55 (8.45 MB/s) - ‘kion_train.zip’ saved [78795385/78795385]



In [10]:
!unzip kion_train.zip

Archive:  kion_train.zip
   creating: kion_train/
  inflating: kion_train/interactions.csv  
  inflating: __MACOSX/kion_train/._interactions.csv  
  inflating: kion_train/users.csv    
  inflating: __MACOSX/kion_train/._users.csv  
  inflating: kion_train/items.csv    
  inflating: __MACOSX/kion_train/._items.csv  


# LOAD DATA

In [11]:
interactions = pd.read_csv('kion_train/interactions.csv')
users = pd.read_csv('kion_train/users.csv')
items = pd.read_csv('kion_train/items.csv')

In [12]:
YEAR_FROM = 1990
STEP_SIZE = 5
bins = [year for year in range(YEAR_FROM, int(items['release_year'].max()) + STEP_SIZE, STEP_SIZE)]

bins = [int(items['release_year'].min())] + bins

items['year_bin'] = pd.cut(items['release_year'],
                           bins=bins, include_lowest=True)

In [13]:
Columns.Datetime = 'last_watch_dt'

In [14]:
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)

In [15]:
items.head(10)

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,year_bin
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ...","(2000.0, 2005.0]"
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео...","(2010.0, 2015.0]"
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг...","(2010.0, 2015.0]"
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю...","(2010.0, 2015.0]"
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж...","(1896.999, 1990.0]"
5,854,film,Северо-Юг,NaN,2015.0,"драмы, русские",Россия,NaN,16.0,NaN,Юрий Грубник,"Алексей Воронин, Алексей Мурашов, Алена Алинин...",Авторский фильм молодого режиссера Юрия Грубни...,"Северо-Юг, 2015, Россия, бандиты, гангстеры, б...","(2010.0, 2015.0]"
6,1468,film,Марья-искусница,NaN,1960.0,"фильмы, сказки, приключения, советские, семейн...",СССР,NaN,6.0,NaN,Александр Роу,"Александр Баранов, Александр Хвыля, Анатолий К...",Два прославленных советских сказочника – писат...,"Марья-искусница, 1960, СССР, преодоление, труд...","(1896.999, 1990.0]"
7,11114,film,Принцесса Лебедь: Пират или принцесса,"The Swan Princess: Princess Tomorrow, Pirate T...",2016.0,"для детей, сказки, полнометражные, зарубежные,...",США,NaN,6.0,Sony Pictures,Ричард Рич,"Брайан Ниссен, Гарднер Джаэс, Грант Дураззо, Д...",Анимационная сказка о непоседливой принцессе Э...,"Принцесса, Лебедь, Пират, или, принцесса, 2016...","(2015.0, 2020.0]"
8,9853,film,Лабиринты прошлого,Todos lo saben,2018.0,"криминал, детективы, драмы, зарубежные, триллеры",Испания,NaN,16.0,NaN,Асгар Фархади,"Барбара Ленни, Инма Куэста, Карла Кампра, Пене...","Испанка Лаура, давно переехавшая в Буэнос-Айре...","Лабиринты, прошлого, 2018, Испания, семейные, ...","(2015.0, 2020.0]"
9,8604,film,Третья попытка,NaN,2013.0,"русские, мелодрамы",Россия,NaN,12.0,NaN,Игорь Мужжухин,"Александр Асташенок, Александр Пашков, Андрей ...","Екатерина Рябова, Александр Асташенок и Алекса...","Третья, попытка, 2013, Россия, любовь, измена,...","(2010.0, 2015.0]"


# Train/Test split

In [16]:
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')

In [17]:
max_date = interactions[Columns.Datetime].max()

In [18]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

In [19]:
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (4985269, 6)
test: (490982, 6)


In [20]:
train.drop(train.query("total_dur < 300").index, inplace=True)

In [21]:
# отфильтруем холодных пользователей из теста
cold_users = set(test[Columns.User]) - set(train[Columns.User])

In [22]:
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

# Добавляем аватаров в обучающую выборку

1. Девушка, которая смотрит только мелодрамы
2. Девочка, смотрит только аниме, хочет уехать в Японию.
3. Фанат Райана Гослинга

In [23]:
dramma_queen = items[(items.genres.isna() == False) & (items.genres.str.contains('мелодрамы'))][:10]
anime_tyan = items[(items.age_rating <= 12) & (items.keywords.str.contains('аниме', case=False))][:15]
gosling_fan = items[(items.actors.isna() == False) & (items.actors.str.contains('Райан Гослинг'))][:10]

new_items = [dramma_queen, anime_tyan, gosling_fan]

In [24]:
max_user_id = train.user_id.max()
avatar_ids = list()

for user_items in new_items:
    max_user_id += 1
    for item_id in user_items.item_id:
        last_watch_dt = f'2022-{str(randint(1,12)).zfill(2)}-{str(randint(1,28)).zfill(2)}'
        if max_user_id not in avatar_ids:
            avatar_ids.append(max_user_id)
        train = train.append({
            'user_id': max_user_id,
            'item_id': item_id,
            'last_watch_dt': last_watch_dt,
            'watched_pct': randint(70, 100),
            'weight': 3
        }, ignore_index=True)

<ipython-input-24-3fb76f32fe70>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append({
<ipython-input-24-3fb76f32fe70>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append({
<ipython-input-24-3fb76f32fe70>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append({
<ipython-input-24-3fb76f32fe70>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append({
<ipython-input-24-3fb76f32fe70>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append({
<ipython-input-24-3fb76f32fe70

In [25]:
avatar_map = pd.DataFrame({"user_id": avatar_ids, "name": ['dramma_queen', 'anime_tyan', 'gosling_fan']})

In [26]:
avatar_map 

,user_id,name
0,1097558,dramma_queen
1,1097559,anime_tyan
2,1097560,gosling_fan


# Обучение моделей

In [27]:
K_RECS = 10
RANDOM_STATE = 1234
NUM_THREADS = 6

In [28]:
dataset = Dataset.construct(
    interactions_df=train
)

/usr/local/lib/python3.9/dist-packages/rectools/dataset/interactions.py:62: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df[Columns.Datetime] = df[Columns.Datetime].astype("datetime64[ns]")


## Подбор гиперпараметров

In [29]:
models = {}

In [30]:
def get_model(model_name, params, n_factors, epochs):
    if model_name == 'light_fm':
        model = LightFMWrapperModel(
            LightFM(
                no_components=n_factors,
                random_state=RANDOM_STATE,
                **params
            ),
            epochs=epochs,
            num_threads=NUM_THREADS,
        )
    elif model_name == 'implicit_als':
        model = ImplicitALSWrapperModel(
            model=AlternatingLeastSquares(
                factors=n_factors,
                random_state=RANDOM_STATE, 
                num_threads=NUM_THREADS,
            ),
            **params
        )
    return model

In [31]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials


models_data = [
    {'name': 'light_fm', 'params': {'loss': 'logistic'}},
    {'name': 'light_fm', 'params': {'loss': 'warp'}},
    {'name': 'implicit_als', 'params': {'fit_features_together': True}},
]
n_factors_var = list(N for N in [4,8,12])

fspace = {
    'model_data': hp.choice('model_data', models_data),
    'n_factors': hp.choice('n_factors', n_factors_var)
}
def train_and_eval(params):
    model_data = params['model_data']
    n_factors = params['n_factors']
    model = get_model(model_data['name'], model_data['params'], n_factors, 10)
    metrics = {f'MAP@{K_RECS}': MAP(k=K_RECS)}

    model.fit(dataset)
    recos = model.recommend(
        users=test[Columns.User].unique(),
        dataset=dataset,
        k=K_RECS,
        filter_viewed=True,
    )
    metric_values = calc_metrics(metrics, recos, test, train)
    suffix = '__'.join([f'{k}_{v}' for k, v in model_data["params"].items()])
    models[f'{model_data["name"]}__{suffix}__n_{n_factors}'] = model

    return {'loss': -metric_values[f'MAP@{K_RECS}'], 'status': STATUS_OK}


trials = Trials()

best = fmin(fn=train_and_eval, space=fspace, algo=tpe.suggest, max_evals=6, trials=trials)

  0%|          | 0/6 [00:00<?, ?trial/s, best loss=?]

 83%|████████▎ | 5/6 [07:58<01:38, 98.41s/trial, best loss: -0.07881307866987335]

100%|██████████| 6/6 [09:18<00:00, 93.08s/trial, best loss: -0.07881307866987335]


In [32]:
model_data = models_data[best['model_data']]
suffix = '__'.join([f'{k}_{v}' for k, v in model_data["params"].items()])
model_key = f'{model_data["name"]}__{suffix}__n_{n_factors_var[best["n_factors"]]}'
print(model_key)

light_fm__loss_warp__n_4


In [33]:
metrics_name = {
    'Precision': Precision,
    'Recall': Recall,
    'MAP': MAP,
}

metrics = {}
for metric_name, metric in metrics_name.items():
    for k in [1, 5, 10]:
        metrics[f'{metric_name}@{k}'] = metric(k=k)

In [34]:
recos = models[model_key].recommend(
    users=test[Columns.User].unique(),
    dataset=dataset,
    k=K_RECS,
    filter_viewed=True,
)
calc_metrics(metrics, recos, test, train)

{'Precision@1': 0.08301689724457191,
 'Recall@1': 0.04224905041931861,
 'Precision@5': 0.051952167255282046,
 'Recall@5': 0.12348164968067045,
 'Precision@10': 0.035181497118882604,
 'Recall@10': 0.16094359114263734,
 'MAP@1': 0.04224905041931861,
 'MAP@5': 0.07286028993904584,
 'MAP@10': 0.07881307866987335}

# Оценка моделей на аватарах

In [35]:
recoms = models[model_key].recommend(
    users=avatar_ids,
    dataset=dataset,
    k=K_RECS,
    filter_viewed=True,
)
recoms = pd.merge(recoms, items, on='item_id')
recoms = pd.merge(recoms, avatar_map, on='user_id')[['name', 'countries', 'title', 'genres', 'age_rating', 'actors', 'keywords']]


## Драма квинн
Модель не так уж плохо справилась, в плане предсказания фильмов, всего 1 мелодрама и 4 драмы

In [36]:
recoms[:10]

,name,countries,title,genres,age_rating,actors,keywords
0,dramma_queen,Россия,Клиника счастья,"драмы, мелодрамы",18.0,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Клиника счастья, Клиника, Счастье, Клиника сча..."
1,dramma_queen,Россия,Хрустальный,"триллеры, детективы",18.0,"Антон Васильев, Николай Шрайбер, Екатерина Оль...","хруст, хрусталь, хруста, хрус, полицейский, пе..."
2,dramma_queen,Россия,Девятаев,"драмы, военные, приключения",12.0,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...","Девятаев, Девятаева, Девят, Девя, Девята, Девя..."
3,dramma_queen,"Великобритания, США",Гнев человеческий,"боевики, триллеры",18.0,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...","ограбление, криминальный авторитет, месть, пер..."
4,dramma_queen,Россия,Секреты семейной жизни,комедии,18.0,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...","брызги крови, кровь, жестокое обращение с живо..."
5,dramma_queen,Россия,Прабабушка легкого поведения,комедии,16.0,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...",", 2021, россия, прабабушка, легкого, поведения"
6,dramma_queen,Россия,Подслушано,"драмы, триллеры",16.0,"Александр Hовиков, Валентина Ляпина, Никита Па...","подслушано, подслушано в контакте, социальная ..."
7,dramma_queen,Россия,Маша,"драмы, триллеры",16.0,"Максим Суханов, Аня Чиповская, Полина Гухман, ...","Фильм Маша, Маша фильм 2021, Смотреть фильм Ма..."
8,dramma_queen,Россия,Афера,комедии,18.0,"Сергей Степин, Игорь Царегородцев, Татьяна Лял...","Афера, Аферисты, Карантин, Пандемия, Карантин ..."
9,dramma_queen,Россия,Белый снег,"драмы, спорт",6.0,"Ольга Лерман, Федор Добронравов, Надежда Марки...","биография, занятие спортом, байопик, чемпионат..."


## Аниме-тян
Всего 1 аниме и 7 мультфильмов. Хотелось бы побольше аниме.


In [37]:
recoms[10:20]

,name,countries,title,genres,age_rating,actors,keywords
10,gosling_fan,Россия,Клиника счастья,"драмы, мелодрамы",18.0,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Клиника счастья, Клиника, Счастье, Клиника сча..."
11,gosling_fan,Россия,Хрустальный,"триллеры, детективы",18.0,"Антон Васильев, Николай Шрайбер, Екатерина Оль...","хруст, хрусталь, хруста, хрус, полицейский, пе..."
12,gosling_fan,Россия,Девятаев,"драмы, военные, приключения",12.0,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...","Девятаев, Девятаева, Девят, Девя, Девята, Девя..."
13,gosling_fan,"Великобритания, США",Гнев человеческий,"боевики, триллеры",18.0,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...","ограбление, криминальный авторитет, месть, пер..."
14,gosling_fan,Россия,Секреты семейной жизни,комедии,18.0,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...","брызги крови, кровь, жестокое обращение с живо..."
15,gosling_fan,Россия,Прабабушка легкого поведения,комедии,16.0,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...",", 2021, россия, прабабушка, легкого, поведения"
16,gosling_fan,Россия,Подслушано,"драмы, триллеры",16.0,"Александр Hовиков, Валентина Ляпина, Никита Па...","подслушано, подслушано в контакте, социальная ..."
17,gosling_fan,Россия,Афера,комедии,18.0,"Сергей Степин, Игорь Царегородцев, Татьяна Лял...","Афера, Аферисты, Карантин, Пандемия, Карантин ..."
18,gosling_fan,Россия,Дуров,документальное,16.0,NaN,"Компьютер, Монитор, Гений, Интервью, Предприни..."
19,gosling_fan,Россия,Сахаров. Две жизни,документальное,16.0,"Алексей Усольцев, Чулпан Хаматова, Агата Супер","Сахаров, Сахарок, Сахар, Сахар сахаров, Сахаро..."


## Фанат Райана Гослинга
В рекомендациях ни 1 фильма с Райаном гослингом. Но что-то схожее с жанром было рекомендовано.

In [38]:
recoms[20:30]

,name,countries,title,genres,age_rating,actors,keywords
20,anime_tyan,США,Ральф против Интернета,"мультфильм, приключения, фантастика, семейное,...",6.0,"Джон Си Райли, Сара Силверман, Галь Гадот, Тар...","видеоигра, мультфильм, продолжение, интернет, ..."
21,anime_tyan,США,Тайна Коко,"мультфильм, фэнтези, приключения",12.0,"Энтони Гонсалес, Гаэль Гарсиа Берналь, Бенджам...","Мексика, гитара, музыкант, скелет, музыка, заг..."
22,anime_tyan,США,Зверополис,"приключения, мультфильм, детективы, комедии",6.0,"Джиннифер Гудвин, Джейсон Бейтман, Идрис Эльба...","аллегория, лев, бегемот, лиса, слон, овца, бел..."
23,anime_tyan,США,Головоломка,"фантастика, мультфильм, комедии",6.0,"Эми Полер, Филлис Смит, Ричард Кайнд, Билл Хей...","мечта, мультфильм, воображаемый друг, начальна..."
24,anime_tyan,США,Монстры на каникулах 3: Море зовёт,"мультфильм, фэнтези, приключения, комедии",6.0,"Энди Сэмберг, Селена Гомес, Кевин Джеймс, Стив...","третья часть, круизное судно, персонаж Дракула..."
25,anime_tyan,США,Холодное сердце II,"фэнтези, мультфильм, музыкальные",6.0,"Идина Мензел, Кристен Белл, Джонатан Грофф, Дж...","королева, магия, королевство, плотина, дух, же..."
26,anime_tyan,США,Вверх,"драмы, мультфильм, приключения, комедии",0.0,"Эдвард Эснер, Кристофер Пламмер, Джордан Нагаи...","разница в возрасте, центральная и южная америк..."
27,anime_tyan,"Австралия, Бельгия",100% волк,"мультфильм, приключения, семейное, фэнтези, ко...",6.0,"Илай Суинделлс, Самара Уивинг, Джай Кортни, Ру...","пудель, подростковая тревога, оборотень, приня..."
28,anime_tyan,США,Суперсемейка 2,"фантастика, мультфильм, приключения",6.0,"Крэйг Т. Нельсон, Холли Хантер, Сара Вауэлл, Х...","семейная пара, продолжение, супергерой, дети, ..."
29,anime_tyan,США,История игрушек 4,"мультфильм, фэнтези, комедии",6.0,"Том Хэнкс, Тим Аллен, Энни Поттс, Тони Хейл, К...","игрушка, дружба, ковбой, история игрушек 4, , ..."
